In [14]:
import sys
sys.path.append("D:\work\models-master")
import tensorflow as tf
import run_classifier
import bert_models
from official.nlp.bert import configs as bert_configs
import json
import tensorflow_hub as hub
from official.nlp.modeling.layers import CRF
import tensorflow_addons  as tfa

In [15]:
input_fn = run_classifier.get_dataset_fn(r"D:\work\models-master\official\nlp\process_dir\ner_data\ner_eval.tf_record",
                                              128, 32, is_training=True)

In [34]:
ds = input_fn()

it = ds.__iter__()

inp = it.next()

x = inp[0] 
y = inp[1]

In [17]:
max_length = 128
num_labels=10
module_url="https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/1"
model_file = r"D:\work\bert_weights"


In [18]:
def BertCrf(bert_file, max_length=512, num_labels=10, hidden_size=768):
    input_word_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="input_word_ids")
    segment_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="segment_ids")
    input_masks = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name="input_masks")
    bert_layer = hub.KerasLayer(model_file, trainable=True)
    _, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])
    emission = tf.keras.layers.Dense(num_labels, activation="tanh")(sequence_output)
    crf = CRF.CRF(num_labels, name='crf_layer')
    decoded_sequence = crf(emission)
    model = tf.keras.Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=decoded_sequence)
    model.compile(optimizer='adam', loss={'crf_layer': crf.get_loss})
    return model

In [19]:
model = BertCrf(model_file, max_length=128, num_labels=11)

In [21]:
def acc_func(y_true, y_pred):
    batch_total_num = 0.0
    batch_correct_num = 0.0
    for i in range(y_true.shape[0]):
        for j in range(y_true.shape[1]):
            if (y_true[i][j] != 0 and y_true[i][j] != 4):
                batch_total_num += 1
                if(y_true[i][j] == y_pred[i][i]):
                    batch_correct_num += 1
    return batch_correct_num, batch_total_num

In [22]:
def compute_acc(ds, model):
    total_num = 0.0
    correct_num = 0.0
    for x, y in ds:
        batch_correct_num, batch_total_num = acc_func(y, model(x))
        total_num += batch_total_num
        correct_num += batch_correct_num
        break
    return correct_num / total_num

In [ ]:
model.fit(x,y,epochs=50)

Epoch 1/50
1/1 [==============================] - 0s 12ms/step - loss: 187.3872
Epoch 2/50
1/1 [==============================] - 0s 3ms/step - loss: 186.0808
Epoch 3/50
1/1 [==============================] - 0s 3ms/step - loss: 184.7671
Epoch 4/50
1/1 [==============================] - 0s 11ms/step - loss: 183.4501
Epoch 5/50
1/1 [==============================] - 0s 3ms/step - loss: 182.1299
Epoch 6/50
1/1 [==============================] - 0s 3ms/step - loss: 180.8178
Epoch 7/50
1/1 [==============================] - 0s 2ms/step - loss: 179.5134
Epoch 8/50
1/1 [==============================] - 0s 2ms/step - loss: 178.2187
Epoch 9/50
1/1 [==============================] - 0s 2ms/step - loss: 176.9323
Epoch 10/50
1/1 [==============================] - 0s 3ms/step - loss: 175.6581
Epoch 11/50
1/1 [==============================] - 0s 3ms/step - loss: 175.1678
Epoch 12/50
1/1 [==============================] - 0s 3ms/step - loss: 567.8224
Epoch 13/50
1/1 [==============================